In [1]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI


In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("No OpenAI API key found. Check your .env file.")

In [3]:
# Set the API key for OpenAI
openai.api_key = openai_api_key

In [5]:
reviews = [
    "Absolutely epic! Building this set was like stepping back into my childhood dreams. The nostalgia and intricate design deserve a five-star rating!",
    "The attention to detail is phenomenal, yet the assembly was a lengthy endeavor. Nonetheless, the final result is undoubtedly worth the time and effort.",
    "Disappointed due to missing pieces in my set, which led to a frustrating building experience. Unfortunately, I can only give it 2 stars.",
    "Incredible design that captivated both my kids and me. We had a blast piecing it together. I highly recommend it for family bonding time.",
    "Its enormous size is overwhelming and consumes a lot of space. In hindsight, I regret the purchase due to its impracticality.",
    "A true masterpiece that resonates with any Star Wars aficionado. The price is steep, but the authenticity and quality justify the cost.",
    "The instructions were perplexing and at times, overwhelming. This set is challenging and requires patience and dedication.",
    "I'm halfway through the build, and it already looks stunning. It's a challenging project, but incredibly rewarding and enjoyable.",
    "This set is quite complex and not really suitable for children. It leans more towards adult Lego enthusiasts due to its intricate details.",
    "Absolutely love it! It's the perfect crowning jewel for my Star Wars collection and a testament to my fandom.",
    "The build is decent, but it falls short of my expectations in terms of interactive features, especially considering the price point.",
    "The set broke apart after just a week, which is disappointing and raises questions about the quality, given the high price.",
    "Building this set felt like a journey through the galaxy, mirroring the epic scope of the Star Wars movies. A truly epic experience.",
    "I found it to be overrated. At the end of the day, it's a collection of bricks that doesn't live up to the hype.",
    "Building this was magical! It feels like I've constructed my very own Death Star. A must-have for fans, and may the force be with all builders!",
    "The instructions were clear and concise, and the pieces fit together seamlessly. It was a thoroughly rewarding building experience.",
    "I was expecting a greater variety of minifigures and felt a bit let down. It's a good set, but it could have been better in this regard.",
    "Took me longer to build than I anticipated, but now it's a stunning centerpiece in my room. It's a remarkable display piece.",
    "It's an okay set, but in my opinion, there are better Lego sets available that offer more value and engagement.",
    "Absolutely stunning once assembled. However, builders should be prepared for a lengthy and demanding construction journey."
]




In [9]:
client = OpenAI()
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=100):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )

    return response.choices[0].message.content


In [25]:
# segment in aspects but also store the aspects as a string in a dictionary
delimiter = "####"
system_message_aspect = f"""
You will be provided with customer reviews of a Lego set. \
The review will be delimited with {delimiter} characters. \
Your task is to segment each review into aspects. \
Output a python dictionary, which has the following format \
review = [review: "...", "aspect1" : "...", "aspect2" : "..." \
add as many aspects as you need for each review \
also add the review itself as a string \
Each aspect should contain a new dimension, functionality, or feature of the Lego set. \
Store the aspects as a string. \
For example, if the review is 'The set is expensive, but it looks great.', \
then the aspects are "price": "The set is expensive" and "appearance" : "but it looks great". \
If possible, use one of the following categories for the aspects: \
Build Quality
Complexity
Design
Educational Value
Entertainment Value
Instruction Clarity
Interactivity
Nostalgia
Price
Size
Age Suitability
Minifigures
Part Availability
Durability
Collectibility
Packaging
Assembly Time
Replay Value

If none of the above categories are suitable, you can use your own categories. \
If you are unable to segment the review into aspects, output an empty string. \
Only ouput the dictionary, with nothing else. \
"""

review_1 = "Absolutely epic! Building this set was like stepping back into my childhood dreams. The nostalgia and intricate design deserve a five-star rating!"
messages = [
        {'role': 'system', 'content': system_message_aspect},
        {'role': 'user', 'content': f"{delimiter}{review_1}{delimiter}"}
]

aspects_review1 = get_completion_from_messages(messages)
print(aspects_review1)

{'review': 'Absolutely epic! Building this set was like stepping back into my childhood dreams. The nostalgia and intricate design deserve a five-star rating!', 'Nostalgia': 'Building this set was like stepping back into my childhood dreams', 'Design': 'The nostalgia and intricate design deserve a five-star rating'}


In [21]:
# convert the dictionary to a string
import json 

aspects_review1_string = json.dumps(aspects_review1)

In [26]:
# analyze the sentiment of each aspect
system_message_sentiment = f"""
You will be provided with a python dictionary of customer reviews of a Lego set. \
Each dictionary contains the whole review as well as their different aspects. \
The review will be delimited with {delimiter} characters. \
Your task is to analyze the sentiment of each aspect of the review individually. \
Output a python dictionary, which has the following format \
review = [review: "...", "aspect1" : "...", "aspect2" : "..." \
add as many aspects as you need for each review \
also add the review itself as a string \
The sentiment has to be one of the following: \
Positive
Neutral
Negative
If you are unable to analyze the sentiment of the aspect, output an empty string. \
Only ouput the dictionary, with nothing else. \
"""

messages =  [  
{'role':'system', 'content': system_message_sentiment},   
{'role': 'user', 'content': f"{delimiter}{aspects_review1_string}{delimiter}"}
] 

final_response = get_completion_from_messages(messages)
print(final_response)


{"review": "Absolutely epic! Building this set was like stepping back into my childhood dreams. The nostalgia and intricate design deserve a five-star rating!", "Nostalgia": "Positive", "Design": "Positive"}


In [27]:
#loop through all reviews
final_outputs = []

for review in reviews:
    messages_aspect = [
        {'role': 'system', 'content': system_message_aspect},
        {'role': 'user', 'content': f"{delimiter}{review}{delimiter}"}
    ]
    aspects = get_completion_from_messages(messages_aspect)


In [28]:
# Convert aspects to a string
aspects_string = json.dumps(aspects)

In [30]:
# Step 3: Analyze sentiment for each set of aspects
messages_sentiment = [
    {'role': 'system', 'content': system_message_sentiment},
    {'role': 'user', 'content': f"{delimiter}{aspects_string}{delimiter}"}
]
sentiment_analysis = get_completion_from_messages(messages_sentiment)

# Store the final response
final_outputs.append(sentiment_analysis)

In [32]:
print(final_outputs)


["{'review': 'Absolutely stunning once assembled. However, builders should be prepared for a lengthy and demanding construction journey.', 'Complexity': 'Neutral', 'Design': 'Positive'}"]
